In [27]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
import category_encoders as ce

In [28]:
train_data = pd.read_csv("./data/train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Preprocessing

In [29]:
# removing useless columns
train_data = train_data.drop(columns=['PassengerId', 'Name'])
train_data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,3,male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,1,female,38.0,1,0,PC 17599,71.2833,C85,C
2,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,1,female,35.0,1,0,113803,53.1000,C123,S
4,0,3,male,35.0,0,0,373450,8.0500,NaN,S


In [30]:
# Replace NaN in Age
train_data['Age'] = train_data['Age'].fillna(train_data['Age'].mean())

In [31]:
# Replace NaN in Cabin with Missing
train_data['Cabin'] = train_data['Cabin'].fillna('Missing')

In [32]:
train_data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,3,male,22.0,1,0,A/5 21171,7.2500,Missing,S
1,1,1,female,38.0,1,0,PC 17599,71.2833,C85,C
2,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,Missing,S
3,1,1,female,35.0,1,0,113803,53.1000,C123,S
4,0,3,male,35.0,0,0,373450,8.0500,Missing,S


## train validation split

In [33]:
X = train_data.iloc[:, 1:]
y = train_data.iloc[:, 0]

In [34]:
X_train,X_valid,y_train,y_valid = train_test_split(X,y,test_size=0.2,random_state=42)

In [35]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
331,1,male,45.5,0,0,113043,28.5000,C124,S
733,2,male,23.0,0,0,28425,13.0000,Missing,S
382,3,male,32.0,0,0,STON/O 2. 3101293,7.9250,Missing,S
704,3,male,26.0,1,0,350025,7.8542,Missing,S
813,3,female,6.0,4,2,347082,31.2750,Missing,S


## Encoder

In [36]:
#onehot encoder for sex and embarked column
X_train = pd.concat([
    X_train, 
    pd.get_dummies(X_train['Sex'], prefix='Sex', drop_first=True),
    pd.get_dummies(X_train['Embarked'], prefix='Embarked', drop_first=True)
], axis=1)

# Drop the original 'Sex' and 'Embarked' columns as they are now encoded
X_train.drop(['Sex', 'Embarked'], axis=1, inplace=True)

# Show the result
print(X_train.head())

     Pclass   Age  SibSp  Parch             Ticket     Fare    Cabin  \
331       1  45.5      0      0             113043  28.5000     C124   
733       2  23.0      0      0              28425  13.0000  Missing   
382       3  32.0      0      0  STON/O 2. 3101293   7.9250  Missing   
704       3  26.0      1      0             350025   7.8542  Missing   
813       3   6.0      4      2             347082  31.2750  Missing   

     Sex_male  Embarked_Q  Embarked_S  
331      True       False        True  
733      True       False        True  
382      True       False        True  
704      True       False        True  
813     False       False        True  


In [ ]:
# BaseNencoding for Ticket and Cabin


## Model

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model.fit(X_train, y_train)


## Prediction

In [3]:
test_data = pd.read_csv("./data/test.csv")

In [4]:
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [18]:
predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")